### Stability Analysis with Root Locus Plot

The purpose of controller stability analysis is to determine the range of controller gains between lower $K_{cL}$ and upper $K_{cU}$ limits that lead to a stable controller.

$K_{cL} \le K_c \le K_{cU}$

The principles of stability analysis presented here are general for any linear time-invariant system whether it is for controller design or for analysis of system dynamics. Several characteristics of a system in the Laplace domain can be deduced without transforming a system signal or transfer function back into the time domain. Some of the analysis relies on the roots of the transfer function denominator, also known as poles. The roots of the numerator, also known as zeros, do not affect the stability directly but can potentially cancel an unstable pole to create an overall stable system.

#### Converge or Diverge

A first point of analysis is whether the system converges or diverges. This is determined by analyzing the roots of the denominator of the transfer function. If any of the real parts of the roots of the denominator are positive then the system is unstable. A simple rule to determine whether there are positive real roots is to examine the signs of the polynomial. If there are mixed signs (+ or -) then the system will be unstable because there is at least one positive real root.

See [Stability Analysis](https://apmonitor.com/pdc/index.php/Main/StabilityAnalysis) for additional information on Routh Arrays, Root Locus Plots, and Bode Plots.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import sympy as sym
from IPython import display

class RLPlot:
    def dev(u,σ=3,n=10,early=False):
        '''
        dev(u,σ=3,n=10)
        
        returns 1 when the i-th value of u is more than σ standard deviations
          from the mean of the last n values of u
        
        ut = np.array([t[i] for i in np.where(dev(u) > 0)[0]])
        '''
        if early:
            du = np.array([abs(np.mean(list(u[max(0,i-n):i+1])+\
                [u[0]]*(max(0,i-n)-(i-n)))-u[i])/max(1e-20,\
                np.std(list(u[max(0,i-n):i+1])+[u[0]]*\
                (max(0,i-n)-(i-n)))) > σ for i in range(len(u))])
        else:
            du = np.array([np.logical_and(abs(np.mean(u[max(0,i-n):i+1])\
                -u[i])/max(1e-20,np.std(u[max(0,i-n):i+1])) > σ, i>n ) \
                for i in range(len(u))])
        return du
    
    def Routh(func,K=sym.Symbol('K'),r=100,o=0,res=1000,it=100,func1=lambda x:1,show=False,Ks=[0],dev=dev):
        '''
        Routh(func,K=sym.Symbol('K'),r=100,o=0,res=200,it=100,func1=lambda x:1,show=False)
        
        determines the range of convergence for a gain function in the s domain
        
        gain function denominator = func = lambda x: f(x,K)
        K = sym.Symbol('K')
        r = radius to check around origin o in res+1 steps
        it = maxiumum interations
        gain function numerator = func1 = lambda x: f(x,K) only used to plot the signal
        show = plot the signal
        
        returns the range of convergence
        '''
        x = sym.Symbol('x')
        f = func(x)
        i=0
        c=[]
        while f!=0 and i<it:
            c = [f.subs(x,0)]+c
            f = sym.diff(f/(i+1),x)
            i+=1
        if show:
            from scipy import signal
            f1 = func1(x)
            i=0
            c0=[]
            while f1!=0 and i<it:
                if type(f1) == int or type(f1) == float:
                    c0 = [f1]+c0
                else:
                    c0 = [f1.subs(x,0)]+c0
                f1 = sym.diff(f1/(i+1),x)
                i+=1
            if sym.diff(func(x),K)==0:
                if sym.diff(func1(x),K)==0:
                    plt.plot(*signal.step(signal.TransferFunction(np.array(c0).astype(float),\
                        np.array(c).astype(float))),label=f'{K} = {i}')
                else:
                    for i in Ks:
                        plt.plot(*signal.step(signal.TransferFunction(np.array([j.subs(K,i) \
                            for j in c0]).astype(float),np.array(c).astype(float))),label=f'{K} = {i}')
            elif sym.diff(func1(x),K)==0:
                for i in Ks:
                    plt.plot(*signal.step(signal.TransferFunction(np.array(c0).astype(float),\
                        np.array([j.subs(K,i) for j in c]).astype(float))),label=f'{K} = {i}')
            else:
                for i in Ks:
                    plt.plot(*signal.step(signal.TransferFunction(np.array([j.subs(K,i) \
                        for j in c0]).astype(float),np.array([j.subs(K,i) for j in c]).astype(float))),\
                             label=f'{K} = {i}')
            plt.xlabel('Time')
            plt.legend()
            plt.show()
        c = np.array(c+[0]*(len(c)%2))
        c = c.reshape((int(len(c)/2),2)).T
        i=0
        while c[-1][0]!=0 and i<2*it:
            n = c[-2,1:]-c[-2,0]*c[-1,1:]/c[-1,0]
            c = np.array(list(c)+[list(n)+[0]])
        c = c.T[0]/c[0,0]
        C = np.copy(c)
        k = np.linspace(o-r,o+r,res+1)
        c = np.array([[i.subs(K,j) for i in c] for j in k])
        c = np.array([all(i==abs(i)) for i in c])
        if all(c):
            k = np.array([k[0],k[-1]])
        else:
            k = np.array([[fsolve(lambda k_: [i.subs(K,k) for k in k_],j)[0] \
                           for i in C] for j in k[np.where(dev(c))[0]]])
            if len(k) == 0:
                k = np.array([np.nan]*2)
            elif len(k) == 1:
                k = k[0][np.where(np.array([min([abs(i.subs(K,j)) for i in C[:-1]]) \
                        for j in k[0]])==min(np.array([min([abs(i.subs(K,j)) \
                                for i in C[:-1]]) for j in k[0]])))[0][0]]
                k = np.array([-np.inf]*int(c[0])+[k]+[np.inf]*int(c[-1]))
            else:
                k = k[0,np.where(np.isclose(*k))[0]]
        return np.sort(k)
    
    def Locus(func,K=sym.Symbol('K'),r=10,o=0,res=1000,it=100,show=False,Routh=Routh):
        '''
        Locus(func,K=sym.Symbol('K'),r=10,o=0,res=1000,it=100,show=False)
        
        determines the range of smooth vs oscillatory behavior
           for a gain function in the s domain
        
        gain function denominator = func = lambda x: f(x,K)
        K = sym.Symbol('K')
        r = radius to check around origin o in res+1 steps
        it = maxiumum interations
        show = show the Root Locus Plot (with slider)
        
        returns the range of smooth convergence
        '''
        x = sym.Symbol('x')
        f = func(x)
        if sym.diff(func(x),K) == 0:
            i=0
            c=[]
            while f!=0 and i<it:
                c = [f.subs(x,0)]+c
                f = sym.diff(f/(i+1),x)
                i+=1
            if all(np.isclose(np.roots(c).imag**2,0)):
                out = np.array([-np.inf,np.inf])
                check = np.array([o-r,o+r])
            else:
                out = np.array([np.nan]*2)
                check = np.array([o,o])
        else:
            check = Routh(func,K,r,o,res,it)
        bounds = np.copy(check)
        
        if not all(np.isfinite(check)):
            out = np.copy(check)
            bounds = np.array([o-r,o+r])
        
        if show or (all(np.isfinite(check)) and sym.diff(func(x),K)!=0):
            if sym.diff(func(x),K) != 0:
                i=0
                c=[]
                while f!=0 and i<it:
                    c = [f.subs(x,0)]+c
                    f = sym.diff(f/(i+1),x)
                    i+=1
            if sym.diff(func(x),K) == 0:
                k = np.linspace(*bounds,1+1)
            else:
                k = np.linspace(*bounds,res+1)
            R = np.array([np.roots(np.array([i.subs(K,j) for i in c])) for j in k])
        
        if all(np.isfinite(check)) and sym.diff(func(x),K)!=0:
            R_ = np.array([abs(np.subtract(*np.meshgrid(a,a))) for a in R])
            R_ = [[a[i,i+1:] for i,_ in enumerate(a)] for a in R_]
            R_ = np.array([min(np.array(np.concatenate(a).flat)) for a in R_])
            if all(R_==R_[0]):
                check = [np.nan]*2
            else:
                check = [k[R_==min(R_)],k[R_==min(R_[R_!=min(R_)])]]

            def zero(k):
                R = np.roots(np.array([i.subs(K,k) for i in c]))
                R = abs(np.subtract(*np.meshgrid(R,R)))
                R = [R[i,i+1:] for i,_ in enumerate(R)]
                return min(np.array(np.concatenate(R).flat))
            out = np.array([fsolve(lambda k: [zero(i) for i in k],p)[0] for p in check])
            if np.isclose(*out):
                out = [np.nan]*2

        if show:
            real = []
            imag = []
            for i in R:
                for j in i:
                    real += [j.real]
                    imag += [j.imag]
            def update(k):
                roots = np.roots(np.array([i.subs(K,k) for i in c]))
                plt.figure(figsize=(12,8))
                ax = plt.gca()
                ax.clear()
                plt.plot(real,imag,'ro',label='Roots')
                plt.plot([0,0],[min(imag),max(imag)],'b--',label='Stability Limit')
                plt.plot(roots.real,roots.imag,'ks',label='Roots (K='+str(k)+')')
                plt.legend(); plt.xlabel('Re'); plt.ylabel('Im'); plt.grid()
                #display.clear_output(wait=True)
                plt.show()
            import ipywidgets as wg
            k_slide = wg.FloatSlider(value=o,min=min(bounds),max=max(bounds),\
                                     step=10*(max(bounds)-min(bounds))/res)
            wg.interact(update, k=k_slide)
            print('Locus Root Plot')

        return np.array(out)

In [2]:
import warnings
with warnings.catch_warnings(record=True):
    K = sym.Symbol('K')  # controller gain
    # denominator of closed-loop transfer function
    RLPlot.Locus(lambda x:(x**2+x+1)*(x+1)**2-2*K,K,10,show=True)

interactive(children=(FloatSlider(value=0.0, description='k', max=1.5, min=-2.0, step=0.015), Output()), _dom_…

Locus Root Plot
